In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def N(x):
    return norm.cdf(x, loc=0, scale=1)
def Np(x):
    return norm.pdf(x, loc=0, scale=1)

def d1(S, K, r, s, T):
    return (np.log(S/K) + (r + s ** 2 / 2) * T) / (s * np.sqrt(T))
def d2(S, K, r, s, T):
    return d1(S, K, r, s, T) - s * np.sqrt(T)

def C(S, K, r, s, T):
    return S * N(d1(S, K, r, s, T)) - K * np.exp(-r * T) * N(d2(S, K, r, s, T))
def P(S, K, r, s, T):
    return K * np.exp(-r * T) * N(-d2(S, K, r, s, T)) - S * N(-d1(S, K, r, s, T))

def deltaC(S, K, r, s, T):
    return N(d1(S, K, r, s, T))
def deltaP(S, K, r, s, T):
    return -N(-d1(S, K, r, s, T))

def gammaC(S, K, r, s, T):
    return Np(d1(S, K, r, s, T)) / (S * s * np.sqrt(T))
def gammaP(S, K, r, s, T):
    return gammaC(S, K, r, s, T)

def thetaC(S, K, r, s, T):
    return S * Np(d1(S, K, r, s, T)) * s / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * Np(d2(S, K, r, s, T))
def thetaP(S, K, r, s, T):
    return S * Np(d1(S, K, r, s, T)) * s / (2 * np.sqrt(T)) + r * K * np.exp(r * T) * Np(-d2(S, K, r, s, T))

def vegaC(S, K, r, s, T):
    return S * Np(d1(S, K, r, s, T)) * np.sqrt(T)
def vegaP(S, K, r, s, T):
    return vegaC(S, K, r, s, T)

def rhoC(S, K, r, s, T):
    return K * T * np.exp(-r * T) * N(d2(S, K, r, s, T))
def rhoP(S, K, r, s, T):
    return -K * T * np.exp(-r * T) * N(-d2(S, K, r, s, T))

In [3]:
#This is a vanilla European Option base class
class Euro_Option(object):
    def __init__(self, S, K, r, s, T):
        self.S = S
        self.K = K
        self.r = r
        self.s = s
        self.T = T
        
    #Calculate d1
    def getd1(self):
        d1value = (np.log(self.S/self.K) + (self.r + self.s ** 2 / 2) * self.T) / (self.s * np.sqrt(self.T))
        return d1value
    
    #Calculate d2
    def getd2(self):
        d2value = self.getd1() - self.s * np.sqrt(self.T)
        return d2value

        
# This is a vanilla European call option class derived from base class with attributes and methods to compute value and greeks    
class Euro_Call(Euro_Option):
    #Initialization
    def __init__(self, S, K, r, s, T):
        Euro_Option.__init__(self, S, K, r, s, T)
    
    #Calculate option value based on BS option pricing formula
    def value(self):
        d1 = self.getd1()
        d2 = self.getd2()
        value = self.S * norm.cdf(d1, loc=0, scale=1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2, loc=0, scale=1)
        print('This is a call!')
        return value
    
    #Calculate delta based on BS option pricing formula
    def delta(self):
        d1 = self.getd1()
        deltavalue = norm.cdf(d1, loc=0, scale=1)
        return deltavalue
    
    #Calculate gamma based on BS option pricing formula
    def gamma(self):
        d1 = self.getd1()
        gammavalue = norm.pdf(d1, loc=0, scale=1) / (self.S * self.s * np.sqrt(T))
        return gammavalue
    
    #Calculate vega based on BS option pricing formula
    def vega(self):
        d1 = self.getd1()
        vegavalue = self.S * norm.cdf(d1, loc=0, scale=1) * np.sqrt(self.T)
        return vegavalue
    
    #Calculate theta based on BS option pricing formula
    def theta(self):
        d1 = self.getd1()
        d2 = self.getd2()
        thetavalue = self.S * norm.pdf(d1, loc=0, scale=1) * self.s / (2 * np.sqrt(T)) - self.r * self.K * np.exp(-self.r * self.T) * norm.pdf(d2, loc=0, scale=1)
        return thetavalue

    #Calculate rho based on BS option pricing formula
    def rho(self):
        d2 = self.getd2()
        rhovalue = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2, loc=0, scale=1)
        return rhovalue
   


In [4]:
# This is a vanilla European put option class derived from base class with attributes and methods to compute value and greeks    
class Euro_Put(Euro_Option):
    #Initialization
    def __init__(self, S, K, r, s, T):
        Euro_Option.__init__(self, S, K, r, s, T)
    
    #Calculate option value based on BS option pricing formula
    def value(self):
        d1 = self.getd1()
        d2 = self.getd2()
        value = self.K * np.exp(-(self.r) * self.T) * norm.cdf((-d2), loc=0, scale=1) - self.S * norm.cdf((-d1), loc=0, scale=1)
        print('This is a Put!')
        return value
    
    #Calculate delta based on BS option pricing formula
    def delta(self):
        d1 = self.getd1()
        deltavalue = -norm.cdf(-d1, loc=0, scale=1)
        return deltavalue
    
    #Calculate gamma based on BS option pricing formula
    def gamma(self):
        d1 = self.getd1()
        gammavalue = norm.pdf(d1, loc=0, scale=1) / (self.S * self.s * np.sqrt(T))
        return gammavalue
    
    #Calculate vega based on BS option pricing formula
    def vega(self):
        d1 = self.getd1()
        vegavalue = self.S * norm.cdf(d1, loc=0, scale=1) * np.sqrt(self.T)
        return vegavalue
    
    #Calculate theta based on BS option pricing formula
    def theta(self):
        d1 = self.getd1()
        d2 = self.getd2()
        thetavalue = self.S * norm.pdf(d1, loc=0, scale=1) * self.s / (2 * np.sqrt(T)) + self.r * self.K * np.exp(self.r * self.T) * norm.pdf(-d2, loc=0, scale=1)
        return thetavalue

    #Calculate rho based on BS option pricing formula
    def rho(self):
        d2 = self.getd2()
        rhovalue = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2, loc=0, scale=1)
        return rhovalue


In [5]:
S, K, r, s, T= 100., 90., 0.05, 0.2, 0.1

option0 = Euro_Put(S, K, r, s, T)
option0_value = option0.value()
option0_delta = option0.delta()
option0_gamma = option0.gamma()
option0_vega = option0.vega()
option0_theta = option0.theta()
option0_rho = option0.rho()

print('value = ' + str(option0_value))
print('delta = ' + str(option0_delta))
print('gamma = ' + str(option0_gamma))
print('vega = ' + str(option0_vega))
print('theta = ' + str(option0_theta))
print('rho = ' + str(option0_rho))

This is a Put!
value = 0.0979800331386
delta = -0.0378190348238
gamma = 0.0130171319719
vega = 30.4268337122
theta = 3.01920127995
rho = -0.387988351552


In [7]:
S, K, r, s, T= 100., 90., 0.05, 0.2, 0.1

option1 = Euro_Call(S, K, r, s, T)
option1_value = option1.value()
option1_delta = option1.delta()
option1_gamma = option1.gamma()
option1_vega = option1.vega()
option1_theta = option1.theta()
option1_rho = option1.rho()

print('value = ' + str(option1_value))
print('delta = ' + str(option1_delta))
print('gamma = ' + str(option1_gamma))
print('vega = ' + str(option1_vega))
print('theta = ' + str(option1_theta))
print('rho = ' + str(option1_rho))

This is a call!
value = 10.5468569058
delta = 0.962180965176
gamma = 0.0130171319719
vega = 30.4268337122
theta = 2.19178853804
rho = 8.56712396118


In [6]:
def forward(S, K, r, s, T):
    return -P(S, K, r, s, T) + C(S, K, r, s, T)

def straddle(S, K, r, s, T):
    return C(S, K, r, s, T) + P(S, K, r, s, T)
def straddledelta(S, K, r, s, T):
    return deltaC(S, K, r, s, T) + deltaP(S, K, r, s, T)
def staddlegamma(S, K, r, s, T):
    return gammaC(S, K, r, s, T) + gammaP(S, K ,r ,s, T)
def straddlevega(S, K, r, s, T):
    return vegaC(S, K, r, s, T) + vegaP(S, K ,r ,s ,T)
def straddletheta(S, K, r, s, T):
    return thetaC(S, K, r, s, T) + thetaP(S, K, r, s, T)
def straddlerho(S, K, r, s ,T):
    return rhoC(S, K, r, s, T) + rhoP(S, K, r, s, T)

In [8]:
S = 2550
r = 0.015775
@interact(K = widgets.FloatSlider(min=2200, max=2700, value=2550), 
          s = widgets.FloatSlider(min=0, max=100, value=13), 
          T = widgets.FloatSlider(min=0, max=0.2, value=0.1))

def plot_mtm(K, s, T):
    S = np.linspace(2200, 2900,100)
    plt.plot(S, C(S, K, r, s/100, T))

A Jupyter Widget

In [9]:
S = 2550
r = 0.015775
@interact(K = widgets.FloatSlider(min=2200, max=2700, value=2550), 
          s = widgets.FloatSlider(min=0, max=100, value=13), 
          T = widgets.FloatSlider(min=0, max=0.2, value=0.1))

def plot_mtm(K, s, T):
    S = np.linspace(2200, 2900,100)
    plt.plot(S, deltaC(S, K, r, s/100, T))

A Jupyter Widget

In [10]:
S = 2550
r = 0.015775
@interact(K = widgets.FloatSlider(min=2200, max=2700, value=2550), 
          s = widgets.FloatSlider(min=0, max=100, value=13), 
          T = widgets.FloatSlider(min=0, max=0.2, value=0.1))

def plot_mtm(K, s, T):
    S = np.linspace(2200, 2900,100)
    plt.plot(S, gammaC(S, K, r, s/100, T))

A Jupyter Widget

In [11]:
@interact(K = widgets.FloatSlider(min=2200, max=2700, value=2550), 
          s = widgets.FloatSlider(min=0, max=100, value=13),
          r = widgets.FloatSlider(min=0, max=100, value=0.015),
          T = widgets.FloatSlider(min=0, max=0.2, value=0.1))

def plot_mtm(K, s, r, T):
    plt.figure(figsize=(20,15))
    S = np.linspace(2200, 2900,100)
    plt.subplot(661)
    plt.plot(S, straddle(S, K, r, s/100, T))
    plt.title('Straddle value')
    
    plt.subplot(662)
    plt.plot(S, straddledelta(S, K, r, s/100, T))
    plt.title('Straddle delta')
    
    plt.subplot(663)
    plt.plot(S, staddlegamma(S, K, r, s/100, T))
    plt.title('Straddle gamma')
    
    plt.subplot(664)
    plt.plot(S, straddlevega(S, K, r, s/100, T))
    plt.title('Straddle vega')
    
    plt.subplot(665)
    plt.plot(S, straddletheta(S, K, r, s/100, T))
    plt.title('Straddle theta')
    
    plt.subplot(666)
    plt.plot(S, straddlerho(S, K, r, s/100, T))
    plt.title('Straddle rho')

A Jupyter Widget